In [1]:
# !pip install PyPortfolioOpt

In [2]:
import requests
import pandas as pd
from datetime import datetime as dt
from pypfopt import expected_returns, risk_models, EfficientFrontier
from pypfopt.discrete_allocation import get_latest_prices, DiscreteAllocation

In [3]:
data = pd.read_csv("stock-eight-pillars-202212141141.csv", index_col=0)

In [4]:
pillars_to_count = 7

In [5]:
mask = data.apply(lambda x: x.value_counts()).apply(lambda y: y['✔️'] >= pillars_to_count)
df = data.T.where(mask).dropna()
df

,5-Year P/E Ratio < 22.5,5-Year ROIC > 9%,5-Year Revenue Growth (Mil),5-Year Net Income Growth (Mil),5-Year Shares Outstanding Growth (%),5-Year LTL to FCF < 5,5-Year Free Cash Flow Growth (Mil),5-Year Price to FCF < 22.5
MMM,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌
AOS,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️
ABBV,✔️,✔️,✔️,✔️,❌,✔️,✔️,✔️
ACN,❌,✔️,✔️,✔️,✔️,✔️,✔️,✔️
ADM,✔️,❌,✔️,✔️,✔️,✔️,✔️,✔️
...,...,...,...,...,...,...,...,...
TRV,✔️,❌,✔️,✔️,✔️,✔️,✔️,✔️
URI,✔️,✔️,✔️,✔️,✔️,❌,✔️,✔️
UNH,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️
WY,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️


In [6]:
tickers = df.index.to_list()

In [7]:
import aiohttp
import asyncio


headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                          AppleWebKit/605.1.15 (KHTML, like Gecko) \
                          Chrome/100.0.4896.127 Safari/605.1.15 Firefox/100.0"
          }


def get_price_history(ticker, data):
    try:
        data = data["chart"]["result"][0]
        lbls = [dt.fromtimestamp(i).date() for i in data["timestamp"]]
        rows = data["indicators"]["quote"][0]["close"]
        df = pd.DataFrame(data=rows, index=lbls, columns=[ticker])
    except:
        return
    return df


async def get_response(ticker, session):
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?range=30y&interval=1d"
    async with session.get(url, headers=headers) as response:
        data = await response.json()
        return get_price_history(ticker, data)


async def main(tickers):
    async with aiohttp.ClientSession() as session:
        return pd.concat(
            await asyncio.gather(*[asyncio.create_task(get_response(ticker, session)) for ticker in tickers]),
            axis=1
        )


prices = await main(tickers)
prices.dropna(inplace=True)
prices

,MMM,AOS,ABBV,ACN,ADM,AAP,AAPL,AMAT,ACGL,AZO,...,TROW,TPR,TGT,TXN,TSCO,TRV,URI,UNH,WY,WHR
2014-09-24,144.880005,24.295000,59.049999,80.199997,51.459999,130.619995,25.437500,21.959999,18.176666,508.630005,...,79.690002,37.080002,63.880001,48.810001,61.619999,94.790001,114.139999,88.209999,32.169998,153.820007
2014-09-25,142.479996,24.209999,58.000000,78.980003,50.480000,128.199997,24.467501,21.510000,18.056667,504.619995,...,78.720001,36.279999,62.880001,48.139999,60.650002,93.699997,110.980003,86.180000,31.959999,150.529999
2014-09-26,142.419998,24.250000,59.189999,79.290001,50.869999,130.100006,25.187500,21.740000,18.226667,508.380005,...,79.199997,36.330002,63.150002,48.330002,61.860001,94.400002,112.500000,86.599998,32.320000,150.789993
2014-09-29,142.190002,24.025000,58.520000,79.919998,51.299999,130.639999,25.027500,21.750000,18.240000,507.839996,...,78.519997,36.160000,63.040001,48.259998,61.430000,93.769997,112.519997,86.510002,32.250000,148.880005
2014-09-30,141.679993,23.639999,57.759998,81.320000,51.099998,130.300003,25.187500,21.610001,18.240000,509.660004,...,78.400002,35.610001,62.680000,47.689999,61.509998,93.940002,111.099998,86.250000,31.860001,145.649994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-07,126.349998,59.270000,165.399994,286.690002,93.169998,146.649994,140.940002,106.110001,60.150002,2455.260010,...,121.250000,36.750000,153.759995,173.750000,217.339996,188.050003,351.519989,542.909973,31.690001,142.759995
2022-12-08,126.000000,58.419998,165.990005,292.149994,92.400002,146.589996,142.649994,108.610001,60.560001,2488.899902,...,122.260002,36.930000,154.210007,176.360001,219.520004,187.720001,361.640015,547.919983,31.670000,145.360001
2022-12-09,125.760002,58.299999,163.059998,288.410004,91.879997,145.899994,142.160004,107.339996,60.040001,2454.989990,...,119.830002,36.389999,152.279999,174.259995,218.320007,185.649994,352.630005,539.200012,31.480000,143.809998
2022-12-12,126.849998,58.509998,165.320007,292.339996,93.330002,145.929993,144.490005,109.370003,60.000000,2451.949951,...,120.989998,37.330002,150.029999,177.479996,214.470001,187.110001,360.450012,545.859985,32.320000,147.449997


In [8]:
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
# print(cleaned_weights)
ef.portfolio_performance(verbose=True)

Expected annual return: 22.8%
Annual volatility: 20.8%
Sharpe Ratio: 1.00


(0.22845135597925526, 0.2075331308042113, 1.0044244751259028)

In [9]:
latest_prices = get_latest_prices(prices)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'UNH': 6, 'AAPL': 14, 'ORLY': 2, 'AZO': 1, 'TECH': 4, 'WRB': 4}
Funds remaining: $62.88
